In [1]:
import numpy as np
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
def write_gro(system, file='test.gro'):
    with open(file, 'w') as file:
        file.write(f'!comment\n{system.shape[0]}\n')
        for i in tqdm(range(system.shape[0])):
            file.write('{:5d}{:<5s}{:5s}{:5d}{:8.3f}{:8.3f}{:8.3f}\n'
                       .format(*system.iloc[i][['resNum', 'res', 'atom', 'atomNum', 'x', 'y', 'z']]))
        box = system[['x', 'y', 'z']].max()
        file.write('{} {} {}\n'.format(*box))
        
with open('solvate_box.gro') as file:
    system = []
    for cnt, line in enumerate(file):
        if cnt>1 and len(line.split()) > 3:
            system.append([int(line[:5]), line[5:10].strip(), line[10:15].strip(), int(line[15:20]), float(line[20:28]), float(line[28:36]), float(line[36:44])])

df = pd.DataFrame(system, columns=['resNum', 'res', 'atom', 'atomNum', 'x', 'y', 'z'])


sol = df[df.res == 'SOL'].copy()
system = df[df.res != 'SOL'].copy()

a = (list(range(1, sol.shape[0]//3+1))*3)
a.sort()
a = np.array(a)
sol['id'] = a

max0 = system.x.max()
min0 = system.x.min()
sol_todel = sol[sol.id.isin(set(sol[sol.x.apply(lambda x: min0<x<max0)].id))]
id_todel = sol_todel.id.unique()

In [3]:
np.random.shuffle(id_todel)
n = int(id_todel.shape[0]*0.7)

sol = sol[~sol.id.isin(id_todel[:n])]

df = pd.concat([system, sol])

n = int(df.shape[0]/100000)+1 
numAt = np.arange(1, 100_000)
numAt = np.concatenate([numAt]*n)[:df.shape[0]]
df['atomNum'] = numAt
print(f'I left {sol.shape[0]/3} mols of water.')
write_gro(df, 'del_wat.gro')

I left 22216.0 mols of water.


  0%|          | 0/89482 [00:00<?, ?it/s]